In [1]:
import pandas as pd
import time
import pprint

# Leitura dos Dados

In [2]:
# Dataset read
df = pd.read_csv('data/unpacked/ca-2021-2022.csv', delimiter=';')
df.head()

,Regiao - Sigla,Estado - Sigla,Municipio,Revenda,CNPJ da Revenda,Nome da Rua,Numero Rua,Complemento,Bairro,Cep,Produto,Data da Coleta,Valor de Venda,Valor de Compra,Unidade de Medida,Bandeira
0,S,RS,SAO LEOPOLDO,POSTOS DE COMBUSTIVEIS REDESINOS LTDA,01.731.418/0001-31,AVENIDA FEITORIA,4288,NaN,FEITORIA,93048-000,GASOLINA,01/01/2021,"4,599",NaN,R$ / litro,BRANCA
1,S,RS,SAO LEOPOLDO,POSTOS DE COMBUSTIVEIS REDESINOS LTDA,01.731.418/0001-31,AVENIDA FEITORIA,4288,NaN,FEITORIA,93048-000,ETANOL,01/01/2021,"4,199",NaN,R$ / litro,BRANCA
2,S,RS,SAO LEOPOLDO,POSTOS DE COMBUSTIVEIS REDESINOS LTDA,01.731.418/0001-31,AVENIDA FEITORIA,4288,NaN,FEITORIA,93048-000,GASOLINA ADITIVADA,01/01/2021,"4,799",NaN,R$ / litro,BRANCA
3,S,RS,SAO LEOPOLDO,POSTOS DE COMBUSTIVEIS REDESINOS LTDA,01.731.418/0001-31,AVENIDA FEITORIA,4288,NaN,FEITORIA,93048-000,DIESEL,01/01/2021,"3,499",NaN,R$ / litro,BRANCA
4,S,RS,SAO LEOPOLDO,POSTOS DE COMBUSTIVEIS REDESINOS LTDA,01.731.418/0001-31,AVENIDA FEITORIA,4288,NaN,FEITORIA,93048-000,DIESEL S10,01/01/2021,"3,599",NaN,R$ / litro,BRANCA


# Experimento Banco de Dados Relacional (SQLite)

In [3]:
from sqlalchemy import create_engine, Column, String, Integer, ForeignKey, DateTime, Float
from sqlalchemy.orm import Session

# Use the default method for abstracting classes to tables
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

In [4]:
class Coleta(Base):
    __tablename__ = 'coleta'
    id = Column(Integer, primary_key=True)
    regiao = Column(String(255))
    estado = Column(String(255))
    municipio = Column(String(255))
    revenda = Column(String(255))
    cnpj = Column(String(255))
    rua = Column(String(255))
    numero = Column(String(255))
    complemento = Column(String(255))
    bairro = Column(String(255))
    cep = Column(String(255))
    produto = Column(String(255))
    data = Column(String(255))
    valor_venda = Column(String(255))
    valor_compra = Column(String(255))
    unidade_medida = Column(String(255))
    bandeira = Column(String(255))

In [5]:
# Create DB connection
engine = create_engine("sqlite:///precos.sqlite")
conn = engine.connect()
session = Session(bind=engine)

# Create metadata layer that abstracts our SQL DB
Base.metadata.create_all(engine)

In [6]:
#Create objects
tempo_ini = time.time()
for index, row in df.iterrows():
    session.add(Coleta(
        id = index,
        regiao = row['Regiao - Sigla'],
        estado = row['Estado - Sigla'],
        municipio = row['Municipio'],
        revenda = row['Revenda'],
        cnpj = row['CNPJ da Revenda'],
        rua = row['Nome da Rua'],
        numero = row['Numero Rua'],
        complemento = row['Complemento'],
        bairro = row['Bairro'],
        cep = row['Cep'],
        produto = row['Produto'],
        data = row['Data da Coleta'],
        valor_venda = row['Valor de Venda'],
        valor_compra = row['Valor de Compra'],
        unidade_medida = row['Unidade de Medida'],
        bandeira = row['Bandeira']
    ))
# Commit changes to database
session.commit()
tempo_fim = time.time()
print('### SQL ### ', df.shape[0], 'registros.')
print("Tempo de inserção: {:.2f} segundos".format((tempo_fim - tempo_ini)))

### SQL ###  1336961 registros.
Tempo de inserção: 221.19 segundos


In [7]:
query = """SELECT * FROM coleta AS c WHERE c.regiao = 'SE';"""
tempo_ini = time.time()
result = pd.read_sql_query(query, session.bind)
tempo_fim = time.time()
print('### SQL ### ', result.shape[0], 'registros.')
print("Tempo de consulta: {:.5f} segundos".format((tempo_fim - tempo_ini)))

### SQL ###  654384 registros.
Tempo de consulta: 3.98760 segundos


# Experimento Banco de Dados Documentos (MongoDB)

In [8]:
import pymongo

# Establish connection
conn = "mongodb://localhost:27017"
client = pymongo.MongoClient(conn)

# Create a database
db = client.precos
db.drop_collection(name_or_collection='coleta')

{'nIndexesWas': 1, 'ns': 'precos.coleta', 'ok': 1.0}

In [9]:
#Create objects
tempo_ini = time.time()
db.coleta.insert_many(df.to_dict('records'))
tempo_fim = time.time()
print('### NoSQL ### ', df.shape[0], 'registros.')
print("Tempo de inserção: {:.2f} segundos".format((tempo_fim - tempo_ini)))

### NoSQL ###  1336961 registros.
Tempo de inserção: 37.51 segundos


In [10]:
find = {'Regiao - Sigla':'SE'}
tempo_ini = time.time()
count = db.coleta.count_documents(find)
tempo_fim = time.time()
print('### NoSQL ### ', count, 'registros.')
print("Tempo de consulta: {:.5f} segundos".format((tempo_fim - tempo_ini)))

### NoSQL ###  654384 registros.
Tempo de consulta: 1.07601 segundos
